In [1]:
#-*- coding:utf-8 -*-
# 2_1 某市财政收入预测模型
# 灰色预测：灰色预测是一种对含有不确定因素的系统进行预测的方法，灰色预测通过鉴别系统因素之间发展趋势的相异程度，即进行关联分析，
# 并对原始数据进行生成处理来寻找系统变动的规律，生成有较强规律性的数据序列，然后建立相应的微分方程模型，从而预测事物未来发展趋势的状况。
# 其用等时距观测到的反应预测对象特征的一系列数量值构造灰色预测模型，预测未来某一时刻的特征量，或达到某一特征量的时间。
# 灰色理论建立的是生成数据模型，不是原始数据模型
# 数据生成方式：A：累加生成：通过数列间各时刻数据的依个累加得到新的数据与数列。累加前数列为原始数列，累加后为生成数列。B：累减生成 C：其他
# 优势：是处理小样本数据预测问题的有效工具
import pandas as pd
import numpy as np
from GM11 import GM11 # 引入自己编写的灰色预测函数

inputfile1 = 'data1.csv'

data = pd.read_csv(inputfile1)
data.index = range(1994,2014)
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
1994,3831732,181.54,448.19,7571.00,6212.70,6370241,525.71,985.31,60.62,65.66,120.0,1.029,5321,64.87
1995,3913824,214.63,549.97,9038.16,7601.73,6467115,618.25,1259.20,73.46,95.46,113.5,1.051,6529,99.75
1996,3928907,239.56,686.44,9905.31,8092.82,6560508,638.94,1468.06,81.16,81.16,108.2,1.064,7008,88.11
1997,4282130,261.58,802.59,10444.60,8767.98,6664862,656.58,1678.12,85.72,91.70,102.2,1.092,7694,106.07
1998,4453911,283.14,904.57,11255.70,9422.33,6741400,758.83,1893.52,88.88,114.61,97.7,1.200,8027,137.32
1999,4548852,308.58,1000.69,12018.52,9751.44,6850024,878.26,2139.18,92.85,152.78,98.5,1.198,8549,188.14
2000,4962579,348.09,1121.13,13966.53,11349.47,7006896,923.67,2492.74,94.37,170.62,102.8,1.348,9566,219.91
2001,5029338,387.81,1248.29,14694.00,11467.35,7125979,978.21,2841.65,97.28,214.53,98.9,1.467,10473,271.91
2002,5070216,453.49,1370.68,13380.47,10671.78,7206229,1009.24,3203.96,103.07,202.18,97.6,1.560,11469,269.10
2003,5210706,533.55,1494.27,15002.59,11570.58,7251888,1175.17,3758.62,109.91,222.51,100.1,1.456,12360,300.55


In [2]:
#-*- coding: utf-8 -*-
# 灰色预测函数
def GM11(x0): #自定义灰色预测函数  #该函数覆盖了导入的包的同名函数
    import numpy as np
    x1 = x0.cumsum() #1-AGO序列
    z1 = (x1[:len(x1)-1] + x1[1:])/2.0 #紧邻均值（MEAN）生成序列 # 由常微分方程可知，取前后两个时刻的值的平均值代替更为合理
    # x0[1] = -1/2.0*(x1[1] + x1[0])
    z1 = z1.reshape((len(z1),1))
    B = np.append(-z1, np.ones_like(z1), axis = 1) # (***)
    Yn = x0[1:].reshape((len(x0)-1, 1))
    [[a],[b]] = np.dot(np.dot(np.linalg.inv(np.dot(B.T, B)), B.T), Yn) #计算参数
#     fkplusone = (x1[0]-b/a)*np.exp(-a*k)#时间响应方程 # 由于x0[0] = x1[0]
    f = lambda k: (x1[0]-b/a)*np.exp(-a*(k-1))-(x1[0]-b/a)*np.exp(-a*(k-2)) #还原值 
    delta = np.abs(x0 - np.array([f(i) for i in range(1,len(x0)+1)])) # 残差
    C = delta.std()/x0.std() # 后验比差值
    P = 1.0*(np.abs(delta - delta.mean()) < 0.6745*x0.std()).sum()/len(x0)
    return f, a, b, x0[0], C, P #返回灰色预测函数、a、b、首项、方差比、小残差概率

In [3]:
data.loc[2014] = None
data.loc[2015] = None
h = ['x1', 'x2', 'x3', 'x4', 'x5', 'x7']
P = []
C = []
for i in h:
    gm = GM11(data[i][range(1994, 2014)].as_matrix())
    f = gm[0] ##获得灰色预测函数
    P = gm[-1] # 获得小残差概率
    C = gm[-2] # 获得后验比差值
    data[i][2014] = f(len(data)-1)
    data[i][2015] = f(len(data))
    data[i] = data[i].round(2) # 保留2位小数
    if (C < 0.35 and P > 0.95): # 评测后验差判别
        print '对于模型%s，该模型精度为---好' % i
    elif (C < 0.5 and P > 0.8):
        print '对于模型%s，该模型精度为---合格' % i
    elif (C < 0.65 and P > 0.7):
        print '对于模型%s，该模型精度为---勉强合格' % i
    else:
        print '对于模型%s，该模型精度为---不合格' % i

对于模型x1，该模型精度为---好
对于模型x2，该模型精度为---好
对于模型x3，该模型精度为---好
对于模型x4，该模型精度为---好
对于模型x5，该模型精度为---好
对于模型x7，该模型精度为---好


In [4]:
#保存的表名命名格式为“2_1_2_1k此表功能名称”，是此小节生成的第1张表格，功能为greyPredict：灰色预测

data[h+['y']].to_excel('2_1_2_1greyPredict.xlsx')

In [5]:
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
1994,3831732.00,181.54,448.19,7571.00,6212.70,6370241.0,525.71,985.31,60.62,65.66,120.0,1.029,5321.0,64.87
1995,3913824.00,214.63,549.97,9038.16,7601.73,6467115.0,618.25,1259.20,73.46,95.46,113.5,1.051,6529.0,99.75
1996,3928907.00,239.56,686.44,9905.31,8092.82,6560508.0,638.94,1468.06,81.16,81.16,108.2,1.064,7008.0,88.11
1997,4282130.00,261.58,802.59,10444.60,8767.98,6664862.0,656.58,1678.12,85.72,91.70,102.2,1.092,7694.0,106.07
1998,4453911.00,283.14,904.57,11255.70,9422.33,6741400.0,758.83,1893.52,88.88,114.61,97.7,1.200,8027.0,137.32
1999,4548852.00,308.58,1000.69,12018.52,9751.44,6850024.0,878.26,2139.18,92.85,152.78,98.5,1.198,8549.0,188.14
2000,4962579.00,348.09,1121.13,13966.53,11349.47,7006896.0,923.67,2492.74,94.37,170.62,102.8,1.348,9566.0,219.91
2001,5029338.00,387.81,1248.29,14694.00,11467.35,7125979.0,978.21,2841.65,97.28,214.53,98.9,1.467,10473.0,271.91
2002,5070216.00,453.49,1370.68,13380.47,10671.78,7206229.0,1009.24,3203.96,103.07,202.18,97.6,1.560,11469.0,269.10
2003,5210706.00,533.55,1494.27,15002.59,11570.58,7251888.0,1175.17,3758.62,109.91,222.51,100.1,1.456,12360.0,300.55
